In [1]:
pip install numpy-stl

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install vtk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [3]:
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
!pip install scipy


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
import numpy as np
from stl import mesh

def calculate_tetrahedra_count(mesh_data):
    # Access the vertices and triangles from the mesh
    vertices = mesh_data.vectors.reshape(-1, 3)
    triangles = mesh_data.vectors

    # Initialize a set to store unique tetrahedra
    unique_tetrahedra = set()

    # Iterate over each triangle and form tetrahedra
    for triangle in triangles:
        tetrahedra = np.vstack([triangle, [0, 0, 0]])
        unique_tetrahedra.add(tuple(map(tuple, tetrahedra)))

    # Count the unique tetrahedra
    tetrahedra_count = len(unique_tetrahedra)

    return tetrahedra_count

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\123597_310-711du_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
tetrahedra_count = calculate_tetrahedra_count(mesh_data)
print(f"Number of Tetrahedra: {tetrahedra_count}")


Number of Tetrahedra: 168


In [6]:
import numpy
import stl

def parse_stl(stl_file):

    # Load the STL file
    mesh_data = mesh.Mesh.from_file(stl_file)

    # Extract vertices and faces
    vertices = mesh_data.vectors.reshape((-1, 3))
    faces = np.arange(len(vertices)).reshape((-1, 3))

    return vertices, faces
face,vertices=parse_stl(r"C:\Users\COE\Desktop\stl_1000\122409_307-134d_flat1_prt.stl")
print(len(face))

1428


In [7]:
import vtk

# Load the STL file
reader = vtk.vtkSTLReader()
reader.SetFileName(r"C:\Users\COE\Desktop\stl_1000\122409_307-134d_flat1_prt.stl")
reader.Update()

# Get the number of cells (faces in an STL file)
num_faces = reader.GetOutput().GetNumberOfCells()

# Print the number of faces
print("Number of faces in the STL file:", num_faces)

Number of faces in the STL file: 476


In [8]:
import os
import numpy as np
from stl import mesh
import networkx as nx
from scipy.sparse.csgraph import connected_components

def calculate_connectivity_topology(mesh_data):
    """
    Calculate connectivity and topology features for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - num_connected_components: Number of connected components in the mesh
    """
    G = nx.Graph()
    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    adjacency_matrix = nx.adjacency_matrix(G)
    num_connected_components = connected_components(adjacency_matrix)[0]

    return num_connected_components

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
num_connected_components = calculate_connectivity_topology(mesh_data)
print(f"Number of Connected Components: {num_connected_components}")

Number of Connected Components: 1


In [9]:
from sklearn.preprocessing import normalize

def calculate_vertex_normals_distribution(mesh_data):
    """
    Calculate the distribution of vertex normals for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - normal_distribution: Tuple containing mean and standard deviation of vertex normals
    """
    vertex_normals = normalize(mesh_data.normals)
    normal_distribution = (np.mean(vertex_normals, axis=0), np.std(vertex_normals, axis=0))

    return normal_distribution

# Example usage
stl_file_path ="C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
normal_distribution = calculate_vertex_normals_distribution(mesh_data)
print(f"Normal Distribution Mean: {normal_distribution[0]}")
print(f"Normal Distribution Std: {normal_distribution[1]}")


Normal Distribution Mean: [ 0.000000e+00 -4.069645e-10  8.028291e-02]
Normal Distribution Std: [0.7100716  0.5062636  0.48275307]


In [10]:
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import numpy as np

def calculate_symmetry_planes(stl_file_path):
    # Read STL file using vtkSTLReader
    reader = vtk.vtkSTLReader()
    reader.SetFileName(stl_file_path)

    # Calculate normals
    normals = vtk.vtkPolyDataNormals()
    normals.SetInputConnection(reader.GetOutputPort())
    normals.ConsistencyOn()
    normals.SplittingOff()

    # Check if normals are successfully computed
    if normals.GetOutput().GetNumberOfPoints() == 0:
        print("Error: Unable to compute normals.")
        return None

    # Extract normals as numpy array
    normals_array = vtk_to_numpy(normals.GetOutput().GetPointData().GetNormals())

    # Check if normals_array is not empty
    if len(normals_array) == 0:
        print("Error: Empty normals array.")
        return None

    # Calculate the centroid of the mesh
    centroid = np.mean(normals_array, axis=0)

    # Use vtkTransformFilter to align the normals along the principal axes
    transform = vtk.vtkTransform()
    transform.Translate(-centroid[0], -centroid[1], -centroid[2])

    transformFilter = vtk.vtkTransformFilter()
    transformFilter.SetInputConnection(normals.GetOutputPort())
    transformFilter.SetTransform(transform)

    # Calculate the covariance matrix
    covarianceMatrix = vtk.vtkMatrix4x4()
    vtk.vtkMath.CovarianceMatrix(normals_array, covarianceMatrix)

    # Calculate the eigenvalues and eigenvectors
    eigenvalue, eigenvector = np.linalg.eigh(covarianceMatrix)

    # Use the eigenvector corresponding to the smallest eigenvalue as the symmetry plane normal
    symmetry_plane_normal = eigenvector[:, 0]

    return symmetry_plane_normal

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
symmetry_plane_normal = calculate_symmetry_planes(stl_file_path)

if symmetry_plane_normal is not None:
    print(f"Symmetry Plane Normal:\n{symmetry_plane_normal}")
else:
    print("Symmetry plane calculation failed.")


Error: Unable to compute normals.
Symmetry plane calculation failed.


In [11]:
!pip install vtk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [12]:
import vtk

# Load the STL file
reader = vtk.vtkSTLReader()
reader.SetFileName("C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl")
reader.Update()

# Calculate mean curvature
curve = vtk.vtkCurvatures()
curve.SetInputConnection(reader.GetOutputPort())
curve.SetCurvatureTypeToMean()  # Or Gaussian curvature if needed
curve.Update()

# Access curvature values
dataObject = curve.GetOutput()
curvature_array = dataObject.GetPointData().GetScalars()

print(curvature_array)

vtkDoubleArray (000001E7C5901D40)
  Debug: Off
  Modified Time: 1020
  Reference Count: 2
  Registered Events: (none)
  Name: Mean_Curvature
  Data type: double
  Size: 236
  MaxId: 235
  NumberOfComponents: 1
  Information: 0000000000000000
  Name: Mean_Curvature
  Number Of Components: 1
  Number Of Tuples: 236
  Size: 236
  MaxId: 235
  LookupTable: (none)




In [13]:
import vtk

def calculate_mean_curvature(filename):
  """
  Calculates the mean curvature of each vertex in an STL file.

  Args:
      filename: The path to the STL file.

  Returns:
      A list of tuples, where each tuple contains the vertex coordinates and its mean curvature.
  """

  # Load the STL file
  reader = vtk.vtkSTLReader()
  reader.SetFileName(filename)
  reader.Update()

  # Calculate mean curvature
  curve = vtk.vtkCurvatures()
  curve.SetInputConnection(reader.GetOutputPort())
  curve.SetCurvatureTypeToMean()
  curve.Update()

  # Extract vertex coordinates and curvature values
  dataObject = curve.GetOutput()
  points = dataObject.GetPoints().GetData()
  curvature_array = dataObject.GetPointData().GetScalars()

  # Create a list of tuples
  vertex_curvatures = []
  for i in range(dataObject.GetNumberOfPoints()):
    point = points.GetTuple(i)
    curvature = curvature_array.GetValue(i)
    vertex_curvatures.append((point, curvature))

  return vertex_curvatures

# Example usage
filename = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
vertex_curvatures = calculate_mean_curvature(filename)

for point, curvature in vertex_curvatures:
  print(f"Point: {point}, Curvature: {curvature}")

Point: (-1.0, 2.75, -4.0536370277404785), Curvature: 0.5078292563684507
Point: (-1.0, 2.75, -0.5464702248573303), Curvature: 0.26736958753955686
Point: (-1.0, -2.75, -0.5464702248573303), Curvature: 0.22917393217676302
Point: (-1.0, -2.75, -0.4744054973125458), Curvature: 0.3208435050474682
Point: (-1.0, -2.75, -4.0536370277404785), Curvature: 0.6347865704605633
Point: (-1.375, -2.75, -4.0536370277404785), Curvature: 0.5078292563684507
Point: (-1.375, -2.75, -0.4744054973125458), Curvature: 0.24798326410910485
Point: (-1.375, -2.75, -0.5464702248573303), Curvature: 0.3208435050474682
Point: (-1.375, 2.75, -0.5464702248573303), Curvature: 0.26736958753955686
Point: (-1.0, 2.75, -0.4744054973125458), Curvature: 0.25275217303427994
Point: (-1.375, 2.75, -0.4744054973125458), Curvature: 0.26736958753955686
Point: (-1.375, 2.75, -4.0536370277404785), Curvature: 0.6347865704605633
Point: (-1.0, 2.743151903152466, 2.755664110183716), Curvature: 3.073400043537347
Point: (-1.0, 2.75, 2.625), Cu

In [14]:
def calculate_centroid(mesh_data):
    """
    Calculate the centroid for an STL mesh.

    Parameters:
    - mesh_data: STL mesh data

    Returns:
    - centroid: Centroid coordinates
    """
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)

    return centroid

# Example usage
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
centroid = calculate_centroid(mesh_data)
print(f"Centroid: {centroid}")


Centroid: [-1.1875000e+00  5.2007973e-08  2.4878144e+00]


In [15]:
import os
import numpy as np
from stl import mesh
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import normalize
from sklearn.mixture import GaussianMixture
import networkx as nx
from scipy.sparse.csgraph import connected_components
from sklearn.impute import SimpleImputer
def extract_holes_and_cavities(mesh_data):
    # Implementation for extracting holes and cavities
    # ...
    G = nx.Graph()

    # Add edges based on vertices of each facet
    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)  # Add individual vertices as nodes
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    # Euler characteristic: V - E + F = 2(1 - g - h)
    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)

    # Calculate the genus (g) and number of handles (h)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)
stl_file_path = "C:\\Users\\COE\\Desktop\\stl_1000\\122409_307-134d_flat1_prt.stl"
mesh_data = mesh.Mesh.from_file(stl_file_path)
cavities = extract_holes_and_cavities(mesh_data)
print(f"Cavities: {cavities}")

Cavities: 59


In [16]:
from stl_metadata import Surface,Facet

In [18]:
import os
import numpy as np
from stl import mesh
from sklearn.mixture import GaussianMixture
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import networkx as nx
from scipy.sparse.csgraph import connected_components
import shutil

def calculate_centroid(mesh_data):
    centroid = np.mean(mesh_data.vectors.reshape(-1, 3), axis=0)
    return centroid

def extract_holes_and_cavities(mesh_data):
    G = nx.Graph()

    for facet in mesh_data.vectors:
        for i in range(3):
            edge = tuple(facet[i])
            G.add_node(edge)
            next_i = (i + 1) % 3
            next_edge = tuple(facet[next_i])
            G.add_edge(edge, next_edge)

    euler_characteristic = 2 - len(G.edges) / 2 + len(G.nodes)
    genus = (2 - euler_characteristic) / 2
    handles = abs(euler_characteristic) / 2

    return int(handles)

def collect_features(folder_path):
    features_list = []
    stl_files = [f for f in os.listdir(folder_path) if f.endswith('.stl')]

    for stl_file in stl_files:
        stl_file_path = os.path.join(folder_path, stl_file)
        mesh_data = mesh.Mesh.from_file(stl_file_path)

        centroid = calculate_centroid(mesh_data)
        cavities = extract_holes_and_cavities(mesh_data)
        
        surface = Surface()
        surface.load(stl_file_path)
        surface_area =  surface.get_area()
        bbv = surface.get_bounding_box_volume()
        features_list.append(np.concatenate([
            np.array(centroid),
            np.array([cavities]),
            np.array([surface_area]),
            # np.array([bbv])
        ]))

    features_array = np.array(features_list)

    imputer = SimpleImputer(strategy='mean')
    features_array_imputed = imputer.fit_transform(features_array)

    return features_array_imputed, stl_files

def cluster_with_gmm(X, n_clusters):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    gmm = GaussianMixture(n_components=n_clusters)
    labels = gmm.fit_predict(X_scaled)

    return labels

def create_cluster_folders(base_folder, labels, stl_files):
    for cluster_label in np.unique(labels):
        cluster_folder = os.path.join(base_folder, f'Cluster_{cluster_label}')
        os.makedirs(cluster_folder, exist_ok=True)

        stl_files_cluster = [stl_file for stl_file, label in zip(stl_files, labels) if label == cluster_label]
        for stl_file in stl_files_cluster:
            source_path = os.path.join(folder_path, stl_file)
            destination_path = os.path.join(cluster_folder, stl_file)
            shutil.copy(source_path, destination_path)

if __name__ == "__main__":
    folder_path = "C:\\Users\\COE\\Desktop\\stl_1000"
    output_folder = "C:\\Users\\COE\\Desktop\\GMM_cav_sa_1000"
    min_samples_per_cluster = 5

    # Collect features
    X, stl_files = collect_features(folder_path)

    # Cluster with GMM
    n_clusters = 187
    labels = cluster_with_gmm(X, n_clusters)

    # Create separate folders for each cluster
    create_cluster_folders(output_folder, labels, stl_files)

    # # Display clustering results
    # for stl_file, label, features in zip(stl_files, labels, X):
    #     print(f"File: {stl_file}")
    #     print(f"  Centroid: {features[:3]}")
    #     print(f"  Cavities: {features[3]}")
    #     print(f"  Cluster Label: {label}")
    #     print("")
